In [1]:
from aif360.sklearn.datasets import fetch_bank, fetch_compas, fetch_adult, fetch_german
import pandas as pd
from tqdm.autonotebook import tqdm as tqdm
import random
import numpy as np

### EDA on all Datasets

In [81]:
#X,y,_ = fetch_adult(subset='train', dropna=True)
#X,y = fetch_bank(dropna=False)
X,y = fetch_german(dropna=True, binary_age=True)

In [82]:
X.describe(include='all')

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,other_parties,residence_since,...,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,sex,marital_status
count,1000,1000.000000,1000,1000,1000.000000,1000,1000,1000.000000,1000,1000.000000,...,1000.000000,1000,1000,1000.000000,1000,1000.000000,1000,1000,1000,1000
unique,4,NaN,5,10,NaN,5,5,NaN,3,NaN,...,NaN,3,3,NaN,4,NaN,2,2,2,4
top,no checking,NaN,existing paid,radio/tv,NaN,<100,1<=X<4,NaN,none,NaN,...,NaN,none,own,NaN,skilled,NaN,none,yes,male,single
freq,394,NaN,530,280,NaN,603,339,NaN,907,NaN,...,NaN,814,713,NaN,630,NaN,596,963,690,548
mean,NaN,20.903000,NaN,NaN,3271.258000,NaN,NaN,2.973000,NaN,2.845000,...,35.546000,NaN,NaN,1.407000,NaN,1.155000,NaN,NaN,NaN,NaN
std,NaN,12.058814,NaN,NaN,2822.736876,NaN,NaN,1.118715,NaN,1.103718,...,11.375469,NaN,NaN,0.577654,NaN,0.362086,NaN,NaN,NaN,NaN
min,NaN,4.000000,NaN,NaN,250.000000,NaN,NaN,1.000000,NaN,1.000000,...,19.000000,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN
25%,NaN,12.000000,NaN,NaN,1365.500000,NaN,NaN,2.000000,NaN,2.000000,...,27.000000,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN
50%,NaN,18.000000,NaN,NaN,2319.500000,NaN,NaN,3.000000,NaN,3.000000,...,33.000000,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN
75%,NaN,24.000000,NaN,NaN,3972.250000,NaN,NaN,4.000000,NaN,4.000000,...,42.000000,NaN,NaN,2.000000,NaN,1.000000,NaN,NaN,NaN,NaN


In [83]:
# X.drop(['poutcome', 'contact'], inplace=True, axis=1)
# X.dropna(inplace=True)

In [84]:
#X.drop(['education-num'], inplace=True, axis=1)

In [85]:
#X['income'] = y
#X['deposit'] = ''

In [86]:
# 0: Bad credit, 1:Good credit
X['credit'] = y

In [87]:
# for i in list(X.index):
#     X.loc[i, 'deposit'] = y.loc[i]

In [89]:
X['sex']

     sex     age    foreign_worker
0    male    aged   yes                 male
1    female  young  yes               female
2    male    aged   yes                 male
3    male    aged   yes                 male
4    male    aged   yes                 male
                                       ...  
995  female  aged   yes               female
996  male    aged   yes                 male
997  male    aged   yes                 male
998  male    young  yes                 male
999  male    aged   yes                 male
Name: sex, Length: 1000, dtype: category
Categories (2, object): ['female' < 'male']

In [90]:
#X['race'].value_counts()

In [92]:
# stats = {}
# stats['00'] = []
# stats['01'] = []
# stats['10'] = []
# stats['11'] = []
# for i in X.index:
#     if X.loc[i]['age'] <= 25 and X.loc[i]['deposit'] == 'yes':
#         stats['01'].append(i)
#     elif X.loc[i]['age'] <= 25 and X.loc[i]['deposit'] == 'no':
#         stats['00'].append(i)
#     elif X.loc[i]['age'] > 25 and X.loc[i]['deposit'] == 'yes':
#         stats['11'].append(i)
#     elif X.loc[i]['age'] > 25 and X.loc[i]['deposit'] == 'no':
#         stats['10'].append(i)
# stats = {}
# stats['00'] = []
# stats['01'] = []
# stats['10'] = []
# stats['11'] = []
# for i in X.index:
#     if X.loc[i]['race'] == 'African-American' and X.loc[i]['category'] == 'Recidivated':
#         stats['01'].append(i)
#     elif X.loc[i]['race'] == 'African-American' and X.loc[i]['category'] == 'Survived':
#         stats['00'].append(i)
#     elif X.loc[i]['race'] == 'Caucasian' and X.loc[i]['category'] == 'Recidivated':
#         stats['11'].append(i)
#     elif X.loc[i]['race'] == 'Caucasian' and X.loc[i]['category'] == 'Survived':
#         stats['10'].append(i)

stats = {}
stats['00'] = []
stats['01'] = []
stats['10'] = []
stats['11'] = []
sensitives = {
    'sex':['female', 'male'],
    'foreign_worker': ['yes', 'no'],
    'age': ['young', 'aged']
}
attr = 'sex'
for i in X.index:
    if attr != 'age':
        if X.loc[i][attr] == sensitives[attr][0] and X.loc[i]['credit'] == 'good':
            stats['01'].append(i)
        elif X.loc[i][attr] == sensitives[attr][0] and X.loc[i]['credit'] == 'bad':
            stats['00'].append(i)
        elif X.loc[i][attr] == sensitives[attr][1] and X.loc[i]['credit'] == 'good':
            stats['11'].append(i)
        elif X.loc[i][attr] == sensitives[attr][1] and X.loc[i]['credit'] == 'bad':
            stats['10'].append(i)
    else:
        if i[2] == sensitives[attr][0] and X.loc[i]['credit'] == 'good':
            stats['01'].append(i)
        elif i[2] == sensitives[attr][0] and X.loc[i]['credit'] == 'bad':
            stats['00'].append(i)
        elif i[2] == sensitives[attr][1] and X.loc[i]['credit'] == 'good':
            stats['11'].append(i)
        elif i[2] == sensitives[attr][1] and X.loc[i]['credit'] == 'bad':
            stats['10'].append(i)

In [93]:
for i in stats:
    print(i, len(stats[i]))

00 109
01 201
10 191
11 499


In [94]:
# for i in X.index:
#     X.at[i, 'age'] = i[2]

In [95]:
X.to_csv('/media/data_dump/Mohit/neurips2022_data/credit/raw/original_full.csv', index=False)

In [96]:
trainIndices = []
testIndices = []
for i in stats:
    random.shuffle(stats[i])
    trainIndices.extend(stats[i][:int(len(stats[i])*0.8)])
    testIndices.extend(stats[i][int(len(stats[i])*0.8):])

In [97]:
X.loc[trainIndices].to_csv('/media/data_dump/Mohit/neurips2022_data/credit/raw/original_train.csv', index=False)
X.loc[testIndices].to_csv('/media/data_dump/Mohit/neurips2022_data/credit/raw/original_test.csv', index=False)

In [98]:
#X.to_csv('/media/data_dump/Mohit/neurips2022_data/adult/raw/original_test.csv', index=False)

In [2]:
def createImbalancedDataframeTabFair(csv_path, balance=True, imbalanceFactor=1.0, dataset='adult', betaSplits=False, beta_neg_factor=None, labelFlip=None):
    data = pd.read_csv(csv_path)
    xVals = list(data.index)
    subgroups = {'00':[], '01':[], '10':[], '11':[]}
    for i in xVals:
        if dataset == 'adult':
            # First index (0) -> Female Gender, Second index(0) -> <=50K, (1) -> >50K
            if data.loc[i]['sex'] == 'Female' and data.loc[i]['income'] == '<=50K':
                subgroups['00'].append(i)
            elif data.loc[i]['sex'] == 'Female' and data.loc[i]['income'] == '>50K':
                subgroups['01'].append(i)
            elif data.loc[i]['sex'] == 'Male' and data.loc[i]['income'] == '<=50K':
                subgroups['10'].append(i)
            elif data.loc[i]['sex'] == 'Male' and data.loc[i]['income'] == '>50K':
                subgroups['11'].append(i)
        elif dataset == 'bank':
            # <= 25 -> 0, yes -> 1
            if data.loc[i]['age'] <= 25 and data.loc[i]['deposit'] == 'no':
                subgroups['00'].append(i)
            elif data.loc[i]['age'] <= 25 and data.loc[i]['deposit'] == 'yes':
                subgroups['01'].append(i)
            elif data.loc[i]['age'] > 25 and data.loc[i]['deposit'] == 'no':
                subgroups['10'].append(i)
            elif data.loc[i]['age'] > 25 and data.loc[i]['deposit'] == 'yes':
                subgroups['11'].append(i)
        elif dataset == 'compas':
            if data.loc[i]['race'] == 'African-American' and data.loc[i]['category'] == 'Recidivated':
                subgroups['01'].append(i)
            elif data.loc[i]['race'] == 'African-American' and data.loc[i]['category'] == 'Survived':
                subgroups['00'].append(i)
            elif data.loc[i]['race'] == 'Caucasian' and data.loc[i]['category'] == 'Recidivated':
                subgroups['11'].append(i)
            elif data.loc[i]['race'] == 'Caucasian' and data.loc[i]['category'] == 'Survived':
                subgroups['10'].append(i)
        elif dataset == 'credit':
            if data.loc[i]['sex'] == 'female' and data.loc[i]['credit'] == 'good':
                subgroups['01'].append(i)
            elif data.loc[i]['sex'] == 'female' and data.loc[i]['credit'] == 'bad':
                subgroups['00'].append(i)
            elif data.loc[i]['sex'] == 'male' and data.loc[i]['credit'] == 'good':
                subgroups['11'].append(i)
            elif data.loc[i]['sex'] == 'male' and data.loc[i]['credit'] == 'bad':
                subgroups['10'].append(i)
        elif dataset == 'synthetic':
            if data.loc[i]['sensitive'] == 0 and data.loc[i]['label'] == 1:
                subgroups['01'].append(i)
            elif data.loc[i]['sensitive'] == 0 and data.loc[i]['label'] == 0:
                subgroups['00'].append(i)
            elif data.loc[i]['sensitive'] == 1 and data.loc[i]['label'] == 1:
                subgroups['11'].append(i)
            elif data.loc[i]['sensitive'] == 1 and data.loc[i]['label'] == 0:
                subgroups['10'].append(i)
    if balance == True:
        num_points = min(len(subgroups['00']), len(subgroups['01']), len(subgroups['10']), len(subgroups['11']))
        for i in subgroups:
            subgroups[i] = random.sample(subgroups[i], num_points)
    if dataset == 'adult':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'income'
        flipped_label = '<=50K'
    elif dataset == 'bank':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'deposit'
        flipped_label = 'no'
    elif dataset == 'compas':
        imbalanceGroup = '11'
        beta_neg_group = '10'
        labelFlipIndex = '11'
        label = 'category'
        flipped_label = 'Survived'
    elif dataset == 'credit':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'credit'
        flipped_label = 'bad'
    elif dataset == 'synthetic':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'label'
        flipped_label = 0
    if labelFlip is not None:
        noisyIndices = random.sample(subgroups[labelFlipIndex], int(len(subgroups[labelFlipIndex])*labelFlip))
        for i in noisyIndices:
            data.at[i,label]  = flipped_label
        return data
    else:
        if betaSplits == True:
            subgroups[imbalanceGroup] = random.sample(subgroups[imbalanceGroup], int(len(subgroups[imbalanceGroup])*imbalanceFactor))
            subgroups[beta_neg_group] = random.sample(subgroups[beta_neg_group], int(len(subgroups[beta_neg_group])*beta_neg_factor))
        newDataset = subgroups['00'] + subgroups['01'] + subgroups['10'] + subgroups['11']
        return data.loc[newDataset]

In [3]:
dataset = 'synthetic'
for nu in tqdm(np.arange(0,10)):
    nu = nu/10
    temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_train.csv', balance=False, dataset=dataset, labelFlip=nu)
    temp.to_csv(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/labelBiasDatasets_train/imbalance_{nu}', index=False)
    # Create similarly biased test set
    temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_test.csv', balance=False, dataset=dataset, labelFlip=nu)
    temp.to_csv(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/labelBiasDatasets_test/imbalance_{nu}', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
dataset = 'synthetic'
temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_test.csv', imbalanceFactor=1.0, balance=True, dataset=dataset)
temp.to_csv(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/balanced/balanced_test.csv', index=False)

In [5]:
dataset = 'synthetic'
# for i in tqdm(range(1,101)):
#     fact = i/100
#     temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_train.csv', imbalanceFactor=fact, balance=True, dataset='compas')
#     temp.to_csv('/media/data_dump/Mohit/neurips2022_data/compas/balanced/realDatasets/imbalance_' + str(i), index=False)
for beta_pos in tqdm(np.arange(1,11)):
    beta_pos = beta_pos/10
    for beta_neg in np.arange(1,11):
        beta_neg = beta_neg/10
        # Create train set
        temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_train.csv', imbalanceFactor=beta_pos, balance=False, dataset=dataset, betaSplits=True, beta_neg_factor=beta_neg)
        temp.to_csv(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/betaDatasets/imbalance_{beta_pos}_{beta_neg}', index=False)
        # Create similarly biased test set
        temp = createImbalancedDataframeTabFair(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_test.csv', imbalanceFactor=beta_pos, balance=False, dataset=dataset, betaSplits=True, beta_neg_factor=beta_neg)
        temp.to_csv(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/test_betaDatasets/imbalance_{beta_pos}_{beta_neg}', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

### Testing Classifiers

In [2]:
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.compose import make_column_selector as selector
import os

In [3]:
# https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html
def preprocessDataset(train_path='/media/data_dump/Mohit/neurips2022_data/adult/raw/realDatasets/imbalance_1', test_path='/media/data_dump/Mohit/neurips2022_data/adult/raw/original_test.csv', dataset='adult'):
    dataset_train = pd.read_csv(train_path)
    dataset_test = pd.read_csv(test_path)
    if dataset == 'adult':
        sensitive='sex'
        label='income'
        dropAttrs = ['income', 'sex', 'race']
        labelReplacementDict = {'<=50K':0, '>50K':1}
        sensitiveReplacementDict = {'Female':0, 'Male':1}
        privileged_groups = [{'sex':1}]
        unprivileged_groups = [{'sex': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'bank':
        sensitive = 'age'
        label='deposit'
        dropAttrs = ['age', 'deposit']
        labelReplacementDict = {'no':0, 'yes':1}
        def sensitiveReplacementFunction(x):
            if x <= 25:
                return 0
            else:
                return 1
        privileged_groups = [{'age':1}]
        unprivileged_groups = [{'age': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'compas':
        sensitive = 'race'
        label = 'category'
        dropAttrs = ['race', 'category', 'sex']
        labelReplacementDict = {'Survived':0, 'Recidivated':1}
        sensitiveReplacementDict = {'African-American':0, 'Caucasian':1}
        privileged_groups = [{'race':0}]
        unprivileged_groups = [{'race': 1}]
        favorable_label = 0
        unfavorable_label = 1
    # Separate out labels and sensitive Attributes
    y_train = dataset_train[label]
    z_train = dataset_train[sensitive]
    x_train = dataset_train.drop(columns=dropAttrs)
    y_test = dataset_test[label]
    z_test = dataset_test[sensitive]
    x_test = dataset_test.drop(columns=dropAttrs)
    # Binarize label and sensitive attribute
    y_train.replace(labelReplacementDict, inplace=True)
    y_test.replace(labelReplacementDict, inplace=True)
    if dataset != 'bank':
        z_train.replace(sensitiveReplacementDict, inplace=True)
        z_test.replace(sensitiveReplacementDict, inplace=True)
    else:
        z_train = z_train.apply(sensitiveReplacementFunction)
        z_test = z_test.apply(sensitiveReplacementFunction)
    # Preprocess Data
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)
    numerical_columns = numerical_columns_selector(x_train)
    categorical_columns = categorical_columns_selector(x_train)
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    numerical_preprocessor = StandardScaler()
    preprocessor = ColumnTransformer([
        ('one-hot-encoder', categorical_preprocessor, categorical_columns),
        ('standard_scaler', numerical_preprocessor, numerical_columns)]).fit(x_train)
    tempTrain = preprocessor.transform(x_train)
    tempTest = preprocessor.transform(x_test)
    # Sometimes we get a sparse CSR matrix, sometimes we get a ndarray
    if isinstance(tempTrain, np.ndarray):
        preprocessed_x_train = pd.DataFrame(tempTrain.tolist())
        preprocessed_x_test = pd.DataFrame(tempTest.tolist())
    else:
        preprocessed_x_train = pd.DataFrame(tempTrain.toarray())
        preprocessed_x_test = pd.DataFrame(tempTest.toarray())
    preprocessed_x_train[label] = y_train
    preprocessed_x_test[label] = y_test
    preprocessed_x_train[sensitive] = z_train
    preprocessed_x_test[sensitive] = z_test
    # Convert to AIF format
    binary_train_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_x_train,
                                label_names=[label],
                                protected_attribute_names=[sensitive],
                                unprivileged_protected_attributes=unprivileged_groups)
    binary_test_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_x_test,
                                label_names=[label],
                                protected_attribute_names=[sensitive],
                                unprivileged_protected_attributes=unprivileged_groups)
    return binary_train_dataset, binary_test_dataset

In [4]:
from aif360.algorithms.preprocessing import OptimPreproc, Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult, get_distortion_compas
from aif360.algorithms.inprocessing import GerryFairClassifier, MetaFairClassifier, PrejudiceRemover
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification, EqOddsPostprocessing
import copy

In [5]:
def splitDataset(train_dataset, train_ratio=0.9, dataset='adult'):
    if dataset == 'adult':
        sensitive='sex'
        label='income'
        privileged_groups = [{'sex':1}]
        unprivileged_groups = [{'sex': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'bank':
        sensitive = 'age'
        label='deposit'
        privileged_groups = [{'age':1}]
        unprivileged_groups = [{'age': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'compas':
        sensitive = 'race'
        label = 'category'
        privileged_groups = [{'race':0}]
        unprivileged_groups = [{'race': 1}]
        favorable_label = 0
        unfavorable_label = 1
    train_dataset = train_dataset.convert_to_dataframe()[0]
    subgroupIndices = {
        '00':[],
        '01':[],
        '10':[],
        '11':[]
    }
    for i in list(train_dataset.index):
        if train_dataset.loc[i][sensitive] == 0 and train_dataset.loc[i][label] == 0: 
            subgroupIndices['00'].append(i)
        elif train_dataset.loc[i][sensitive] == 0 and train_dataset.loc[i][label] == 1: 
            subgroupIndices['01'].append(i)
        elif train_dataset.loc[i][sensitive] == 1 and train_dataset.loc[i][label] == 0: 
            subgroupIndices['10'].append(i)
        elif train_dataset.loc[i][sensitive] == 1 and train_dataset.loc[i][label] == 1: 
            subgroupIndices['11'].append(i)
    trainIndices = []
    valIndices = []
    for i in subgroupIndices:
        random.shuffle(subgroupIndices[i])
        trainIndices.extend(subgroupIndices[i][:int(len(subgroupIndices[i])*train_ratio)])
        valIndices.extend(subgroupIndices[i][int(len(subgroupIndices[i])*train_ratio):])
    trainDataset = copy.deepcopy(train_dataset).loc[trainIndices]
    valDataset = copy.deepcopy(train_dataset).loc[valIndices]
    trainDataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=trainDataset,
                                label_names=[label],
                                protected_attribute_names=[sensitive],
                                unprivileged_protected_attributes=unprivileged_groups)
    valDataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=valDataset,
                                label_names=[label],
                                protected_attribute_names=[sensitive],
                                unprivileged_protected_attributes=unprivileged_groups)
    return trainDataset, valDataset

In [77]:
def trainClassifiers(train_dataset, test_dataset, dataset='adult'):
    if dataset == 'adult':
        sensitive='sex'
        label='income'
        privileged_groups = [{'sex':1}]
        unprivileged_groups = [{'sex': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'bank':
        sensitive = 'age'
        label='deposit'
        privileged_groups = [{'age':1}]
        unprivileged_groups = [{'age': 0}]
        favorable_label = 1
        unfavorable_label = 0
    elif dataset == 'compas':
        sensitive = 'race'
        label = 'category'
        privileged_groups = [{'race':0}]
        unprivileged_groups = [{'race': 1}]
        favorable_label = 0
        unfavorable_label = 1
    results={}
    # Preprocessing Algorithms
    results['pre'] = {}
    # Skipping Disparate Impact remover since it requires the conditional distributions to be same between training and testing sets
    #dis_impact_remover = DisparateImpactRemover(sensitive_attribute='sex')
    #transformed_train_dataset = dis_impact_remover.fit_transform(train_dataset)
    #transformed_test_dataset = dis_impact_remover.fit_transform(test_dataset)
    for i in ['rew', 'optPre']:
        if i == 'rew':
            preProc = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        elif i == 'optPre':
            preProc = OptimPreproc()
        preProc.fit(train_dataset)
        new_train_dataset = preProc.transform(train_dataset)
        cls = LogisticRegression().fit(new_train_dataset.features[:,:-1], new_train_dataset.labels, sample_weight=new_train_dataset.instance_weights)
        new_preds_test = cls.predict(test_dataset.features[:,:-1])
        pred_test_set = copy.deepcopy(test_dataset)
        pred_test_set.labels = new_preds_test
        results['pre'][i] = ClassificationMetric(test_dataset, pred_test_set, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    # Inprocessing Algorithms
    results['in'] = {}
    for i in ['gerry_dp', 'gerry_eo', 'meta_fair_dp', 'meta_fair_eo', 'prej_remover', 'exp_grad_dp', 'exp_grad_eo']:
        if i == 'gerry_dp':
            inProc = GerryFairClassifier(fairness_def='SP')
        elif i == 'gerry_eo':
            inProc = GerryFairClassifier(fairness_def='FP')
        elif i == 'meta_fair_dp':
            inProc = MetaFairClassifier(sensitive_attr=sensitive, type='sr')
        elif i == 'meta_fair_eo':
            inProc = MetaFairClassifier(sensitive_attr=sensitive, type='fdr')
        elif i == 'prej_remover':
            inProc = PrejudiceRemover(sensitive_attr=sensitive, class_attr=label)
        elif i == 'exp_grad_dp':
            inProc = ExponentiatedGradientReduction(LogisticRegression(), constraints='DemographicParity')
        elif i == 'exp_grad_eo':
            inProc = ExponentiatedGradientReduction(LogisticRegression(), constaints='EqualizedOdds')
        inProc.fit(train_dataset)
        pred_test_set = inProc.predict(test_dataset)
        results['in'][i] = ClassificationMetric(test_dataset, pred_test_set, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    # Postprocessing Algorithms
    results['post'] = {}
    new_train_set, new_val_set = splitDataset(train_dataset, train_ratio=0.8, dataset=dataset)
    base_classifier = LogisticRegression()
    base_classifier.fit(new_train_set.features[:,:-1], new_train_set.labels)
    preds_val = base_classifier.predict(new_val_set.features[:,:-1])
    scores_val = base_classifier.predict_proba(new_val_set.features[:,:-1])
    pred_val_set = copy.deepcopy(new_val_set)
    pred_val_set.labels = preds_val
    pred_val_set.scores = scores_val[:,1]
    test_dataset.scores = base_classifier.predict_proba(test_dataset.features[:, :-1])[:,1]
    for i in ['eq', 'cal_eq', 'reject']:
        if i == 'eq':
            postProc = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        elif i == 'cal_eq':
            postProc = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        elif i == 'reject':
            postProc = RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, metric_name="Average odds difference")
        postProc.fit(new_val_set, pred_val_set)
        pred_test_set = postProc.predict(test_dataset)
        results['post'][i] = ClassificationMetric(test_dataset, pred_test_set, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    return results

In [66]:
#train_dataset, test_dataset = preprocessDataset(dataset='bank', train_path='/media/data_dump/Mohit/neurips2022_data/bank/raw/realDatasets/imbalance_1', test_path='/media/data_dump/Mohit/neurips2022_data/bank/raw/original_test.csv')
train_dataset, test_dataset = preprocessDataset('/media/data_dump/Mohit/neurips2022_data/bank/raw/betaDatasets/imbalance_0.1_0.1', '/media/data_dump/Mohit/neurips2022_data/bank/raw/original_test.csv', 'bank')
#train_dataset, test_dataset = preprocessDataset(dataset='compas', train_path='/media/data_dump/Mohit/neurips2022_data/compas/raw/realDatasets/imbalance_1', test_path='/media/data_dump/Mohit/neurips2022_data/compas/raw/original_test.csv')
# dataset = 'adult'
# for i in os.listdir(f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/realDatasets/'):
#     train_dataset, test_dataset = preprocessDataset(dataset=dataset, train_path=f'/media/data_dump/Mohit/neurips2022_data/{dataset}/balanced/realDatasets/{i}', test_path=f'/media/data_dump/Mohit/neurips2022_data/{dataset}/raw/original_test.csv')

In [67]:
new_train_set, new_val_set = splitDataset(train_dataset, 0.8, 'bank')

In [68]:
from sklearn.linear_model import LogisticRegression
base_classifier = LogisticRegression()
base_classifier.fit(new_train_set.features[:,:-1], new_train_set.labels)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


LogisticRegression()

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset.labels, base_classifier.predict(test_dataset.features[:,:-1]))

0.8824074074074074

In [70]:
preds_val = base_classifier.predict(new_val_set.features[:,:-1])
preds_scores = base_classifier.predict_proba(new_val_set.features[:,:-1])
pred_val_set = copy.deepcopy(new_val_set)
pred_val_set.labels = preds_val
pred_val_set.scores = preds_scores[:,1]

In [71]:
# sensitive='sex'
# label='income'
# privileged_groups = [{'sex':1}]
# unprivileged_groups = [{'sex': 0}]
# favorable_label = 1
# unfavorable_label = 0
## Bank variables
sensitive = 'age'
label='deposit'
privileged_groups = [{'age':1}]
unprivileged_groups = [{'age': 0}]
favorable_label = 1
unfavorable_label = 0
#exp_grad = ExponentiatedGradientReduction(estimator=LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False)
#temp_cls = DisparateImpactRemover(sensitive_attribute='sex')
#temp_cls = OptimPreproc(OptTools)
#temp_cls = GerryFairClassifier()
#temp_cls = MetaFairClassifier(sensitive_attr='sex', type='sr')
#temp_cls = PrejudiceRemover(sensitive_attr='sex', class_attr='income')
#temp_cls = RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, metric_name="Average odds difference")
#temp_cls = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
temp_cls = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

In [72]:
temp_cls = temp_cls.fit(new_val_set, pred_val_set)

In [73]:
#preds_test = temp_cls.predict(test_dataset)
#preds_test_set = copy.deepcopy(test_dataset)
#preds_test_set.labels = preds_test

In [74]:
# privileged_groups = [{'sex':1}]
# unprivileged_groups = [{'sex': 0}]
# favorable_label = 1
# unfavorable_label = 0
sensitive = 'age'
label='deposit'
privileged_groups = [{'age':1}]
unprivileged_groups = [{'age': 0}]
favorable_label = 1
unfavorable_label = 0

In [75]:
#BinaryLabelDatasetMetric(test_dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups).statistical_parity_difference()
test_dataset.scores = base_classifier.predict_proba(test_dataset.features[:, :-1])[:,1]
pred_test_set = temp_cls.predict(test_dataset)
temp = ClassificationMetric(test_dataset, pred_test_set, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

In [76]:
temp.statistical_parity_difference()

0.0019647180439068546

In [64]:
ClassificationMetric(test_dataset, test_dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups).statistical_parity_difference()

0.12198569175874825

In [65]:
temp.accuracy()

0.5002314814814814